Vamos a comenzar con una prueba de scraping de imágenes y contenido, en donde tendrás que robotear imágenes de un sitio de baterías y realizar lo siguiente:

* Almacenarlas localmente
* Renombrar las imágenes dándoles el siguiente formato Marca-OEM.jpg o en su defecto Marca-CodigoPieza.jpg o el formato de imagen nativo png etc. Ejemplo: EXIDE- EK508.jpg
* Subir las mismas a un servidor web, hosting o Google Drive
* Crear automáticamente un archivo de Excel con el nombre de cada imagen dado anteriormente juntamente con su URL o ubicación de la misma en el servidor

In [1]:
#Importamos librerías necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
from urllib.parse import urlsplit

In [2]:
#Importamos el contenido de la pagina principal
url = 'https://www.exidegroup.com/es/es/battery-finder/browse-all'
response = requests.get(url)

In [3]:
#Definimos objeto de BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
#Buscamos tablas
tables = soup.find_all('table')

In [5]:
#Guardamos las tablas como dataframes
dfs = []                                                #Definimos lista de dataframes
for table in tables:                                    #Para cada una de las tablas
    df = pd.read_html(str(table))[0]                    #Transformamos en dataframe
    df.columns = df.columns.droplevel(0)                #Dropeamos primer header
    df.drop(columns=df.columns[0],inplace=True)         #Dropeamos columna 1 que no aporta información
    for i in range(df.shape[0]):
        try:
            int(df['Capacidad (Ah)'][i])
        except:
            df.drop(index=i,inplace=True)               #Dropeamos filas innecesarias
    df.reset_index(drop=True,inplace=True)              #Reseteamos indices
    dfs.append(df)                                      #Agregamos a lista de dataframes

In [6]:
#Vemos que sean 4 dataframes
len(dfs)

4

In [7]:
#Definimos cada dataframe con un nombre según su aplicación
cars = dfs[0]
trucks_buses = dfs[1]
marine = dfs[2]
motorbikes = dfs[3]

In [8]:
#Concatenamos todo en un solo dataframe
bateries = pd.concat([cars,trucks_buses,marine,motorbikes],ignore_index=True)
bateries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Referencia comercial  294 non-null    object
 1   Capacidad (Ah)        294 non-null    object
 2   CCA (A) (A)           294 non-null    object
 3   Tamaño recipiente     294 non-null    object
 4   La (mm)               294 non-null    object
 5   An (mm)               294 non-null    object
 6   Al (mm)               294 non-null    object
 7   Talón                 294 non-null    object
 8   Esquema (Polaridad)   294 non-null    object
 9   Tipo de terminal      294 non-null    object
 10  MCA (A)               58 non-null     object
 11  WH                    164 non-null    object
dtypes: object(12)
memory usage: 27.7+ KB


In [9]:
#Hay baterías que tiene el caracter "/" dentro de su nombre. Las identificamos.
bateries[bateries['Referencia comercial'].str.contains('/')]

,Referencia comercial,Capacidad (Ah),CCA (A) (A),Tamaño recipiente,La (mm),An (mm),Al (mm),Talón,Esquema (Polaridad),Tipo de terminal,MCA (A),WH
130,EV1300/24,50,-,G77,307,170,216,B0,ETN 1,Female M8,-,1300
134,EV3800/36,100,-,H52,520,269,221,B0,ETN 4,Female M8,-,3800


In [10]:
#Definimos url de cada batería. Reemplazamos "/" por "--" según lo visto en la página
bateries['Fin_Url'] = bateries['Referencia comercial'].str.replace('/','--')
bateries.iloc[130:135]

,Referencia comercial,Capacidad (Ah),CCA (A) (A),Tamaño recipiente,La (mm),An (mm),Al (mm),Talón,Esquema (Polaridad),Tipo de terminal,MCA (A),WH,Fin_Url
130,EV1300/24,50,-,G77,307,170,216,B0,ETN 1,Female M8,-,1300,EV1300--24
131,EV640,50,-,D31,308,168,211,B0,ETN 1,Female M8,-,640,EV640
132,EV1250,96,-,L05,355,176,190,B13,ETN 0,EN taper post,-,1250,EV1250
133,EV1300,100,-,D31,308,168,211,B0,ETN 1,Female M8,-,1300,EV1300
134,EV3800/36,100,-,H52,520,269,221,B0,ETN 4,Female M8,-,3800,EV3800--36


In [11]:
bateries.head()

,Referencia comercial,Capacidad (Ah),CCA (A) (A),Tamaño recipiente,La (mm),An (mm),Al (mm),Talón,Esquema (Polaridad),Tipo de terminal,MCA (A),WH,Fin_Url
0,EK508,50,800,G34,260,173,206,B7,ETN 9,EN taper post,NaN,NaN,EK508
1,EK600,60,680,L02,242,175,190,B13,ETN 0,EN taper post,NaN,NaN,EK600
2,EK620,62,680,L02,242,175,190,B13,ETN 0,EN taper post,NaN,NaN,EK620
3,EK700,70,760,L03,278,175,190,B13,ETN 0,EN taper post,NaN,NaN,EK700
4,EK720,72,760,L03,278,175,190,B13,ETN 0,EN taper post,NaN,NaN,EK720


Acabamos de terminar la preparación de la tabla con todas las baterías. Ahora procederemos a recolectar las imágenes y almacenarlas localmente.

In [12]:
#Vamos a ver un ejemplo y después aplicarlo a todas las baterías
url_bat = 'https://www.exidegroup.com/es/es/battery/EK508'
response_bat = requests.get(url_bat)

In [13]:
#Definimos objeto de BeautifulSoup
soup_bat = BeautifulSoup(response_bat.content,'html.parser')

In [14]:
#Obtenemos todas las imágenes que haya
images = soup_bat.find_all('img')

In [15]:
#Vemos cuantas son
len(images)

15

In [16]:
#Descargamos todas las imágenes de manera local
for i,img in enumerate(images):
    source = img.get('src')                                     #Tomamos la url de la imagen
    urllib.request.urlretrieve(source, "image"+str(i)+".jpg")   #La guardamos temporariamente como "image1.jpg","image2.jpg",etc.

ValueError: unknown url type: '/es/themes/exide/images/logo-exide.svg'

In [17]:
#La celda anterior falla. Vemos cual puede ser el problema
for i,img in enumerate(images):
    source = img.get('src')                                         #Tomamos la url de la imagen
    try:
        urllib.request.urlretrieve(source, "image"+str(i)+".jpg")   #Si podemos la guardamos como "image1.jpg","image2.jpg",etc.
    except:
        print(i)            #Si no marcamos cual imagen no se puede

0
1
2
4
5
8
9
10
11
12
13


In [18]:
#Solamente se descargan las imágenes 3, 6, 7 y 14. Vemos que tienen de distintas las fuentes.
for i,img in enumerate(images):
    source = img.get('src')         #Tomamos la url de la imagen
    print(i,': ',source)            #Imprimimos la fuente con cada una de las imágenes

0 :  /es/themes/exide/images/logo-exide.svg
1 :  /es/sites/default/files/2022-11/EXIDE-LOGO.png
2 :  /es/sites/default/files/2022-11/Tudor-logo.png
3 :  https://www.exidegroup.com/es/sites/default/files/country-icons/spain.svg
4 :  /es/themes/exide/images/logo-exide.svg
5 :  /es/themes/exide/images/logo-exide.svg
6 :  https://www.exidegroup.com/es/sites/default/files/country-icons/spain.svg
7 :  https://www.exidegroup.com/es/sites/default/files/2022-11/EXIDE-LOGO.png
8 :  /es/sites/default/files/sharepoint/1TtTpfJLXpRE390T53JeuSMnL-Edk2fJ5iTbNKQ_8l0-large.png
9 :  /es/sites/default/files/sharepoint/tEA7uISY4MCSXV1NcA3IrMcTG-l4fqW-BXAmBDCe6dY-large.png
10 :  /es/sites/default/files/sharepoint/5z7U2AkjDNBKE1Uif74Sik6KD8S5lwXwWHMShwU9niA-large.png
11 :  /es/sites/default/files/sharepoint/2ukj6VpEwZBWFAE-3_hIW3h1I-tcwwga4_sk-5NFyC8-large.png
12 :  /es/sites/default/files/sharepoint/BKuJxWLBB6vd9JigPltQJTssn9v9h-BgSzpS0h0IX-o-large.png
13 :  /es/sites/default/files/inline-images/exide_techn

In [ ]:
#Vemos que las que fallan tienen los paths relativos nomás. Le agregamos lo que corresponde así se guardan las 15 imágenes
for i,img in enumerate(images):
    source = img.get('src')                                     #Tomamos la url de la imagen
    if source[0] != 'h':
        source = 'https://www.exidegroup.com' + source          #Agregamos lo que falta del path en caso de que sea necesario
    urllib.request.urlretrieve(source, "image"+str(i)+".jpg")   #La guardamos como "image1.jpg","image2.jpg",etc.

La imagen que necesitamos es la 8 nomas. La probamos con un par de baterías más para asegurarnos que ese siempre sea el caso.

In [20]:
#Copiamos las celdas de arriba
url_bat = 'https://www.exidegroup.com/es/es/battery/ED851T'
response_bat = requests.get(url_bat)
soup_bat = BeautifulSoup(response_bat.content,'html.parser')
images = soup_bat.find_all('img')
for i,img in enumerate(images):
    source = img.get('src')                                     #Tomamos la url de la imagen
    if source[0] != 'h':
        source = 'https://www.exidegroup.com' + source          #Agregamos lo que falta del path en caso de que sea necesario
    urllib.request.urlretrieve(source, "image"+str(i)+".jpg")   #La guardamos como "image1.jpg","image2.jpg",etc.

In [22]:
#Copiamos las celdas de arriba
url_bat = 'https://www.exidegroup.com/es/es/battery/ELT9B'
response_bat = requests.get(url_bat)
soup_bat = BeautifulSoup(response_bat.content,'html.parser')
images = soup_bat.find_all('img')
for i,img in enumerate(images):
    source = img.get('src')                                     #Tomamos la url de la imagen
    if source[0] != 'h':
        source = 'https://www.exidegroup.com' + source          #Agregamos lo que falta del path en caso de que sea necesario
    urllib.request.urlretrieve(source, "image"+str(i)+".jpg")   #La guardamos como "image1.jpg","image2.jpg",etc.

Perfecto. Solo vamos a guardar la imagen 8 entonces de cada batería y la guardamos con el código solicitado.

In [25]:
for url_end in bateries['Fin_Url'].values:
    url_bat = 'https://www.exidegroup.com/es/es/battery/'+url_end           #Definimos la página
    response_bat = requests.get(url_bat)                                    #Obtenemos la respuesta
    soup_bat = BeautifulSoup(response_bat.content,'html.parser')            #Definimos objeto de BeautifulSoup
    images = soup_bat.find_all('img')                                       #Hallamos todas las imágenes
    source = images[8].get('src')                                           #Tomamos el path de la imagen que nos interesa
    if source[0] != 'h':                                                    #En caso de que sea necesario
        source = 'https://www.exidegroup.com' + source                      #Agregamos lo que falta del path
    urllib.request.urlretrieve(source, "imagenes/EXIDE- "+url_end+".jpg")   #Guardamos la imagen con el nombre solicitado

Se guardaron todas las imágenes de manera correcta. Ahora vamos a subirlas a Google Drive, dentro de una carpeta pública, usando la API de Google Drive. Para setear inicialmente los permisos se crea el archivo quickstart.py según lo que se explica en la documentación y se ejecuta. Automáticamente se genera el archivo "token.json" para gestión de permisos.

In [12]:
#Importamos librerías de Google Drive API
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [13]:
#Definimos los permisos
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = Credentials.from_authorized_user_file('token.json', SCOPES)

In [14]:
#Creamos el servicio
service = build('drive', 'v3', credentials=creds)

In [15]:
#Importamos los datos de archivos y carpetas que están en la unidad
results = service.files().list(pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

In [16]:
#Obtenemos el Id de la carpeta pública creada en Drive. Comprobamos que esté bien.
for item in items:
    if item['name'] == 'Carpeta publica':
        folder_id = item['id']
        print(folder_id)

1jqYaTlzWvpx8RpKnTttBJTHh6xMO9r0x


In [21]:
#Subimos los archivos
for bat in bateries['Fin_Url'].values:
    upload = MediaFileUpload('imagenes/EXIDE- '+bat+'.jpg', resumable=False)
    file = service.files().create(body={'name': 'EXIDE- '+bat+'.jpg','parents': [folder_id]},
                                  media_body=upload,fields='id').execute()

Finalmente armamos el excel solicitado con la url y el nombre del archivo.

In [27]:
#Importamos los datos de archivos y carpetas que están en la carpeta
from googleapiclient.errors import HttpError
try:
    results_images = service.files().list(q=f"'{folder_id}' in parents and trashed = false",
                                fields="nextPageToken, files(id, name)",pageSize=300).execute()
    images_data = results_images.get('files', [])
except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')

In [28]:
#Armamos dataframe
images_data_df = pd.DataFrame(images_data)
images_data_df

,id,name
0,1w_cuGb98OEUnd4KvAEUnCvG7XRDptc6Z,EXIDE- 4901.jpg
1,1iZDzMKW2vlSe4vjr3HpET-kQRXe1k7OQ,EXIDE- 4900.jpg
2,1yJJz9SG_HSCk7b0EomZJyb9CASKj2TP1,EXIDE- EB30L-B.jpg
3,1fR-bKJHUOqY9GcC7xzZkASJbWI8_NIaG,EXIDE- E60-N30L-B.jpg
4,1LxvHd_64IZiSuiygQI0OsH7hgKZBFPvg,EXIDE- E60-N30L-A.jpg
...,...,...
289,1vUbZdokUuuQFoOr5PVUZkJanRdRRUd-g,EXIDE- EK720.jpg
290,1_1hQTpaqjaQxeamvuetDFc4lG-Nb-xTf,EXIDE- EK700.jpg
291,1VKtgSDZ7jzuQllwZbRlwzcph6NnUZrhI,EXIDE- EK620.jpg
292,11PGBw0L9FcE3eIFOZwytUgQTYlfBfsUw,EXIDE- EK600.jpg


In [29]:
#Agregamos columna con la url
images_data_df['url']='https://drive.google.com/uc?id='+images_data_df['id']
images_data_df

,id,name,url
0,1w_cuGb98OEUnd4KvAEUnCvG7XRDptc6Z,EXIDE- 4901.jpg,https://drive.google.com/uc?id=1w_cuGb98OEUnd4...
1,1iZDzMKW2vlSe4vjr3HpET-kQRXe1k7OQ,EXIDE- 4900.jpg,https://drive.google.com/uc?id=1iZDzMKW2vlSe4v...
2,1yJJz9SG_HSCk7b0EomZJyb9CASKj2TP1,EXIDE- EB30L-B.jpg,https://drive.google.com/uc?id=1yJJz9SG_HSCk7b...
3,1fR-bKJHUOqY9GcC7xzZkASJbWI8_NIaG,EXIDE- E60-N30L-B.jpg,https://drive.google.com/uc?id=1fR-bKJHUOqY9Gc...
4,1LxvHd_64IZiSuiygQI0OsH7hgKZBFPvg,EXIDE- E60-N30L-A.jpg,https://drive.google.com/uc?id=1LxvHd_64IZiSui...
...,...,...,...
289,1vUbZdokUuuQFoOr5PVUZkJanRdRRUd-g,EXIDE- EK720.jpg,https://drive.google.com/uc?id=1vUbZdokUuuQFoO...
290,1_1hQTpaqjaQxeamvuetDFc4lG-Nb-xTf,EXIDE- EK700.jpg,https://drive.google.com/uc?id=1_1hQTpaqjaQxea...
291,1VKtgSDZ7jzuQllwZbRlwzcph6NnUZrhI,EXIDE- EK620.jpg,https://drive.google.com/uc?id=1VKtgSDZ7jzuQll...
292,11PGBw0L9FcE3eIFOZwytUgQTYlfBfsUw,EXIDE- EK600.jpg,https://drive.google.com/uc?id=11PGBw0L9FcE3eI...


In [32]:
#Creamos archivo excel solicitado
images_data_df[['name','url']].to_excel('baterias.xlsx',index=False)